# Comprobar la calidad de la predicción


Dividir el dataframe en dos particiones:
 - El conjunto de entrenamiento: tiene que tener la mayoría de las filas (75-80%)
 - El conjunto de test: el resto de las filas (20-25%)
 
Vamos a buscar los k vecinos más cercanos en el conjunto de entrenamiento (*train*), en ningún caso en el de test 

Luego, por cada fila del conjunto de *test*, vamos a calcular la predicción, y la vamos a comparar con el resultado real. En este conjunto, añadiremos una fila, que llamaremos *predicted_price* y la compararemos con los valores de *price* reales. 

In [1]:
import pandas as pd
import numpy as np
import math

valencia_df = pd.read_csv('airbnb_valencia.csv')
# corregir los datos de la columna price
valencia_df['price'] = valencia_df['price'].str.replace(',', '').str.replace('$', '', regex=False).astype('float')
# barajar los datos - es importante hacerlo ANTES de crear los conjuntos de entrenamiento y test
np.random.seed(2)
valencia_df = valencia_df.loc[np.random.permutation(len(valencia_df))]

In [2]:
math.trunc(len(valencia_df)*0.8)

4820

In [3]:
# generamos los dos conjuntos
train_df = valencia_df.iloc[0:math.trunc(len(valencia_df)*0.8)].copy()
test_df = valencia_df.iloc[math.trunc(len(valencia_df)*0.8):].copy()

In [4]:
train_df.shape

(4820, 74)

In [5]:
test_df.shape

(1205, 74)

In [6]:
def predict_price(mi_capacidad):
    temp_df = train_df.copy()
    temp_df['distance'] = temp_df['accommodates'].apply(lambda x: np.abs(mi_capacidad - x))
    return temp_df.sort_values('distance').iloc[0:5]['price'].mean()

In [7]:
test_df['predicted_price'] = test_df['accommodates'].apply(predict_price)

In [8]:
test_df[['name', 'price', 'predicted_price']]

,name,price,predicted_price
2469,BEACH AREA ★ COZY APARTMENT WITH PRIVATE TERRACE,61.0,96.2
3617,Habitación disponible para fallas!,31.0,90.4
859,Mercado Central X,131.0,96.2
12,Lovely Central Market Plaza Apt.,80.0,84.0
4852,"Apartamento lujo con piscina, parque y cércala...",200.0,121.6
...,...,...,...
1099,PISO ACOGEDOR EN EL CENTRO (Pl. Pilar),89.0,121.6
2514,Loft 2 independiente Piscina Centro Valencia,64.0,96.2
3606,Habitación amplia y luminosa !,36.0,84.0
5704,"FV-Nice Flat 2 rooms, good located,wifi high s...",73.0,92.2


In [9]:
# esto no valdría porque los resultados negativos y positivos se anulan
(test_df['price'] - test_df['predicted_price']).mean()

-6.589709543568465

In [10]:
# vamos a coger la media de las desviaciones pero en valor absoluto
(np.abs(test_df['price'] - test_df['predicted_price'])).mean()

43.709377593361

MAE = MEDIUM ABSOLUTE ERROR (ERROR MEDIO ABSOLUTO)

In [22]:
# A continuación, en aras de penalizar valores muy alejados, podemos hacer la media del cuadrado de las diferencias

mse_accommodates = ((test_df['price'] - test_df['predicted_price'])**2).mean()


# Entrenamos otro modelo 



In [12]:
valencia_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6025 entries, 103 to 2575
Data columns (total 74 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            6025 non-null   int64  
 1   listing_url                                   6025 non-null   object 
 2   scrape_id                                     6025 non-null   int64  
 3   last_scraped                                  6025 non-null   object 
 4   name                                          6025 non-null   object 
 5   description                                   5848 non-null   object 
 6   neighborhood_overview                         3458 non-null   object 
 7   picture_url                                   6025 non-null   object 
 8   host_id                                       6025 non-null   int64  
 9   host_url                                      6025 non-null  

In [13]:
def predict_price_bedrooms(n_bedrooms):
    temp_df = train_df.copy()
    temp_df['distance'] = temp_df['bedrooms'].apply(lambda x: np.abs(n_bedrooms - x))
    return temp_df.sort_values('distance').iloc[0:5]['price'].mean()

In [14]:
test_df['predicted_price_bedrooms'] = test_df['bedrooms'].apply(predict_price_bedrooms)

In [15]:
test_df[['name', 'price', 'predicted_price', 'predicted_price_bedrooms']]

,name,price,predicted_price,predicted_price_bedrooms
2469,BEACH AREA ★ COZY APARTMENT WITH PRIVATE TERRACE,61.0,96.2,99.8
3617,Habitación disponible para fallas!,31.0,90.4,70.0
859,Mercado Central X,131.0,96.2,68.2
12,Lovely Central Market Plaza Apt.,80.0,84.0,68.2
4852,"Apartamento lujo con piscina, parque y cércala...",200.0,121.6,70.0
...,...,...,...,...
1099,PISO ACOGEDOR EN EL CENTRO (Pl. Pilar),89.0,121.6,99.8
2514,Loft 2 independiente Piscina Centro Valencia,64.0,96.2,68.2
3606,Habitación amplia y luminosa !,36.0,84.0,68.2
5704,"FV-Nice Flat 2 rooms, good located,wifi high s...",73.0,92.2,99.8


In [16]:
mse_bedrooms = ((test_df['price']- test_df['predicted_price_bedrooms'])**2).mean()

In [18]:
mse_bedrooms

7925.540946058091

## Raiz del error cuadrático medio

- MSE - nos permite identificar qué modelo ha rendido mejor en términos relativos, pero nos nos ayuda a entender si es lo suficientemente bueno en general. 

- RMSE = √MSE - las unidaddes son nuestra unidad base (€)

In [20]:
rmse_bedrooms = mse_bedrooms**(1/2)

In [24]:
rmse_bedrooms

89.02550727773524

In [26]:
rmse_accommodates = mse_accommodates**0.5

In [52]:
rmse_accommodates

81.00495328331185

In [54]:
def predict_price_k15(mi_capacidad):
    temp_df = train_df.copy()
    temp_df['distance'] = temp_df['accommodates'].apply(lambda x: np.abs(mi_capacidad - x))
    return temp_df.sort_values('distance').iloc[0:15]['price'].mean()

In [55]:
test_df['predicted_accommodates_k15'] = test_df['accommodates'].apply(predict_price_k15)

In [58]:
((test_df['predicted_accommodates_k15'] - test_df['price'])**2).mean()**0.5 #RMSE

79.2081679383004

# Comparar MAE y RMSE


In [47]:
diferencias_one = pd.Series([5,10,5,10,5,10,5,10,5,10,5,10,5,10,5,10,5,10])
diferencias_two = pd.Series([5,0,5,0,5,0,5,0,5,0,5,0,5,0,5,0,0,1000])

In [41]:
mae_one = diferencias_one.sum()/len(diferencias_one)
rmse_one = ((diferencias_one**2).sum()/len(diferencias_one))**0.5

In [36]:
mae_one

7.5

In [38]:
rmse_one

7.905694150420948

In [48]:
mae_two = diferencias_two.sum()/len(diferencias_two)
rmse_two = ((diferencias_two**2).sum()/len(diferencias_two))**0.5

In [49]:
mae_two

57.77777777777778

In [50]:
rmse_two

235.72582944316193